In [166]:
import re

class Rule:
    def __init__(self, name):
        self.name = name
    def apply(self, data):
        pass

class FieldRule(Rule):
    type = "field"
    def __init__(self, name, pattern):
        super(FieldRule, self).__init__(name)
        self.pattern = re.compile(pattern)
    def isValid(self, data):
        preProcessedData = self.preProcess(data)
        result = self.pattern.fullmatch(preProcessedData)
        print(result)
        if result != None:
            return self.postProcess(preProcessedData)
        else: 
            return None
    def preProcess(self, data):
        return data;
    def postProcess(self, preProcessedData):
        return preProcessedData;
        
class BlankField(FieldRule):
    def __init__(self, name):
        super(BlankField, self).__init__(name, '^[\s]{0,}$')
        
class WholeNumberField(FieldRule):
    def __init__(self, name, size=0):
        super(WholeNumberField, self).__init__(name, "".join(["^\d{0,", str(size), "}$"]))
    def preProcess(self, data):
        return data.lstrip('0')
    def postProcess(self, preProcessedData):
        return int(preProcessedData)
    
class DecimalNumberField(FieldRule):
    def __init__(self, name, characteristic=10, mantissa=2):
        super(DecimalNumberField, self).__init__(name, "".join(["^\d{0,",str(characteristic),"}\.?\d","{0,}$"]))
        self.characteristic = characteristic
        self.mantissa = mantissa
    def postProcess(self, preProcessedData):
        return round(float(preProcessedData), self.mantissa)
    
class DateField(FieldRule):
    def patterns(self, pattern):
        cases = {
            "mmddyyyy" : '^(0[1-9]|1[012])[- /.]?(0[1-9]|[12][0-9]|3[01])[- /.]?(19|20)\d\d$',
            "ddmmyyyy" : '^(0[1-9]|[12][0-9]|3[01])[- /.]?(0[1-9]|1[012])[- /.]?(19|20)\d\d$',
        }
        return cases[pattern]
    def __init__(self, name, checkFormat="mmddyyyy"):
        print(self.patterns(checkFormat))
        super(DateField, self).__init__(name, self.patterns(checkFormat))
        
## Tests        


In [132]:
emailField = FieldRule("email rule", '^[a-z0-9]+[\._]?[a-z0-9]+[@]\w+[.]\w{2,3}$')
print(emailField.isValid("rambo.2@cool.is"))

<re.Match object; span=(0, 15), match='rambo.2@cool.is'>
rambo.2@cool.is


In [133]:
blankField = BlankField("blank rule")
print(blankField.isValid(""))
print(blankField.isValid("  "))
print(blankField.isValid("as"))

<re.Match object; span=(0, 0), match=''>

<re.Match object; span=(0, 2), match='  '>
  
None
None


In [149]:
wholeNumberField = WholeNumberField("whole number rule", 5)
print(wholeNumberField.isValid("23333"))
print(wholeNumberField.isValid("233338"))
print(wholeNumberField.isValid("0000023333"))

<re.Match object; span=(0, 5), match='23333'>
23333
None
None
<re.Match object; span=(0, 5), match='23333'>
23333


In [143]:
decimalNumberField = DecimalNumberField("decimal number rule", 10, 2)
print(decimalNumberField.isValid("345.268"))
print(decimalNumberField.isValid("345.2100a"))
print(decimalNumberField.isValid("345a.2100"))
print(decimalNumberField.isValid("00000345.2100"))
print(decimalNumberField.isValid("3450000000.2157"))
print(decimalNumberField.isValid("34500000007.2157"))

<re.Match object; span=(0, 7), match='345.268'>
345.27
None
None
None
None
<re.Match object; span=(0, 13), match='00000345.2100'>
345.21
<re.Match object; span=(0, 15), match='3450000000.2157'>
3450000000.22
None
None


In [169]:
dateField = DateField("date format rule", "mmddyyyy")
print(dateField.isValid('03-21-2019'))
print(dateField.isValid('03212019'))
print(dateField.isValid('03/21/2019'))
print(dateField.isValid('03.21.2019'))
print(dateField.isValid('03-21-2019'))
print(dateField.isValid('13212019'))
print(dateField.isValid('03/32/2019'))
print(dateField.isValid('03.21.22019'))
dateField = DateField("date format rule", "ddmmyyyy")
print(dateField.isValid('21032019'))
print(dateField.isValid('21/03/2019'))
print(dateField.isValid('21/03/2019'))
print(dateField.isValid('21/03/2019'))

^(0[1-9]|1[012])[- /.]?(0[1-9]|[12][0-9]|3[01])[- /.]?(19|20)\d\d$
<re.Match object; span=(0, 10), match='03-21-2019'>
03-21-2019
<re.Match object; span=(0, 8), match='03212019'>
03212019
<re.Match object; span=(0, 10), match='03/21/2019'>
03/21/2019
<re.Match object; span=(0, 10), match='03.21.2019'>
03.21.2019
<re.Match object; span=(0, 10), match='03-21-2019'>
03-21-2019
None
None
None
None
None
None
^(0[1-9]|[12][0-9]|3[01])[- /.]?(0[1-9]|1[012])[- /.]?(19|20)\d\d$
<re.Match object; span=(0, 8), match='21032019'>
21032019
<re.Match object; span=(0, 10), match='21/03/2019'>
21/03/2019
<re.Match object; span=(0, 10), match='21/03/2019'>
21/03/2019
<re.Match object; span=(0, 10), match='21/03/2019'>
21/03/2019
